In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [8]:
data = pd.read_csv('data/gemstone.csv')

In [9]:
data.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [10]:
data = data.drop(['id'],axis=1)

In [11]:
data.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [12]:
X = data.drop(['price'],axis=1)

In [15]:
X.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77


In [16]:
y = data['price']

In [18]:
y.head()

0    13619
1    13387
2     2772
3      666
4    14453
Name: price, dtype: int64

In [19]:
numerical_features = [x for x in X.columns if X.dtypes[x] != 'object']
categorical_features = [x for x in X.columns if X.dtypes[x] == 'object']

In [20]:
numerical_features

['carat', 'depth', 'table', 'x', 'y', 'z']

In [21]:
categorical_features

['cut', 'color', 'clarity']

In [24]:
print(X['cut'].unique())
print(X['color'].unique())
print(X['clarity'].unique())

['Premium' 'Very Good' 'Ideal' 'Good' 'Fair']
['F' 'J' 'G' 'E' 'D' 'H' 'I']
['VS2' 'SI2' 'VS1' 'SI1' 'IF' 'VVS2' 'VVS1' 'I1']


In [32]:
# custom ranking for each ordinal varible
cut_categories = ['Fair','Good','Very Good','Premium','Ideal']
color_categories = ['J','I','H','G','F','E','D']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [37]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [45]:
numerical_pipeline = Pipeline(
    steps=[
        ('Imputer',SimpleImputer()),
        ('Scaler',StandardScaler())
    ]
)

In [44]:
categorical_pipeline = Pipeline(
    steps=[
        ('Imputer',SimpleImputer(strategy='most_frequent')),
        ('Encoder',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories]))
    ]
)

In [46]:
preprocessor = ColumnTransformer(
    [
        ('numerical_pipeline',numerical_pipeline,numerical_features),
        ('categorical_pipeline',categorical_pipeline,categorical_features)
    ]
)

In [58]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=30)

In [61]:
preprocessor.fit_transform(X_train)

array([[-0.97543926, -0.84960654, -0.12153081, ...,  4.        ,
         1.        ,  5.        ],
       [ 0.2351953 ,  1.83363716, -0.12153081, ...,  1.        ,
         5.        ,  2.        ],
       [ 0.49461699,  0.81585507,  0.39980029, ...,  3.        ,
         3.        ,  4.        ],
       ...,
       [ 0.45138004,  1.55606023, -0.6428619 , ...,  1.        ,
         3.        ,  2.        ],
       [ 0.66756478, -1.77486298,  1.44246248, ...,  4.        ,
         3.        ,  4.        ],
       [ 0.25681377,  0.81585507, -0.12153081, ...,  4.        ,
         3.        ,  2.        ]])

In [60]:
preprocessor.transform(X_test)

array([[-0.56468825, -0.94213218, -0.6428619 , ...,  3.        ,
         5.        ,  3.        ],
       [-0.17555571,  1.00090636, -0.12153081, ...,  2.        ,
         2.        ,  2.        ],
       [-1.06191316,  0.2607012 , -0.12153081, ...,  4.        ,
         2.        ,  7.        ],
       ...,
       [-0.19717419, -3.34779894,  1.44246248, ...,  3.        ,
         0.        ,  3.        ],
       [-0.82410994, -0.20192702, -0.12153081, ...,  4.        ,
         3.        ,  2.        ],
       [ 2.61322747, -0.75708089,  1.44246248, ...,  3.        ,
         0.        ,  3.        ]])

In [62]:
preprocessor.get_feature_names_out()

array(['numerical_pipeline__carat', 'numerical_pipeline__depth',
       'numerical_pipeline__table', 'numerical_pipeline__x',
       'numerical_pipeline__y', 'numerical_pipeline__z',
       'categorical_pipeline__cut', 'categorical_pipeline__color',
       'categorical_pipeline__clarity'], dtype=object)

In [63]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [64]:
X_train.head()

,numerical_pipeline__carat,numerical_pipeline__depth,numerical_pipeline__table,numerical_pipeline__x,numerical_pipeline__y,numerical_pipeline__z,categorical_pipeline__cut,categorical_pipeline__color,categorical_pipeline__clarity
0,-0.975439,-0.849607,-0.121531,-1.042757,-1.080970,-1.123150,4.0,1.0,5.0
1,0.235195,1.833637,-0.121531,0.318447,0.279859,0.485354,1.0,5.0,2.0
2,0.494617,0.815855,0.399800,0.570855,0.606458,0.673737,3.0,3.0,4.0
3,-1.018676,0.260701,0.921131,-1.214034,-1.244270,-1.195605,3.0,3.0,3.0
4,-0.953821,-0.664555,-0.642862,-1.069801,-1.044681,-1.094168,4.0,0.0,5.0


In [65]:
X_test.head()

,numerical_pipeline__carat,numerical_pipeline__depth,numerical_pipeline__table,numerical_pipeline__x,numerical_pipeline__y,numerical_pipeline__z,categorical_pipeline__cut,categorical_pipeline__color,categorical_pipeline__clarity
0,-0.564688,-0.942132,-0.642862,-0.429765,-0.464061,-0.500036,3.0,5.0,3.0
1,-0.175556,1.000906,-0.121531,-0.042137,-0.028595,0.036132,2.0,2.0,2.0
2,-1.061913,0.260701,-0.121531,-1.304180,-1.298703,-1.268060,4.0,2.0,7.0
3,0.970223,-0.201927,1.963794,1.048629,0.996563,0.978049,3.0,3.0,3.0
4,-0.932202,-1.312235,0.399800,-1.006699,-0.990248,-1.065186,3.0,5.0,4.0


In [53]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

In [54]:
import numpy as np
def evaluate_model(true,predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = np.sqrt(mse)
    r2_square = r2_score(true,predicted)
    
    return mae,mse,rmse,r2_square

In [55]:
models = {
    'Linear Regression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet()
}

In [56]:
models.keys()

dict_keys(['Linear Regression', 'Lasso', 'Ridge', 'ElasticNet'])

In [57]:
models.values()

dict_values([LinearRegression(), Lasso(), Ridge(), ElasticNet()])

In [82]:
models_list = []
r2_list = []


In [88]:

def best_model(r2_list,models_list):
    ind = 0
    max_element = r2_list[0]
    for i in range(1,len(r2_list)):
        if r2_list[i]>max_element:
            max_element = r2_list[i]
            ind = i
            
    return (models_list[ind],max_element)

In [93]:
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train)
    
    y_pred = model.predict(X_test)
    
    mae,mse,rmse,r2_square = evaluate_model(y_test,y_pred)
    
    print(list(models.keys())[i])
    models_list.append(list(models.keys())[i])
    
    print("Model Training Performance")
    print(f'Root Mean Squared Error : {rmse}')
    print(f'Mean Absolute Error : {mae}')
    print(f'R2 Score : {r2_square*100}')
    
    r2_list.append(r2_square)
    print('='*50)
    print('\n')
    
m,score = best_model(r2_list,models_list)
print(f'Best Model found : {m}, R2_Score : {score}')

Linear Regression
Model Training Performance
Root Mean Squared Error : 1013.9047094344005
Mean Absolute Error : 674.0255115796836
R2 Score : 93.68908248567512


Lasso
Model Training Performance
Root Mean Squared Error : 1013.8723151049944
Mean Absolute Error : 675.2036880701619
R2 Score : 93.68948574778251


Ridge
Model Training Performance
Root Mean Squared Error : 1013.9058997761063
Mean Absolute Error : 674.056513229593
R2 Score : 93.68906766741328


ElasticNet
Model Training Performance
Root Mean Squared Error : 1513.9140350229763
Mean Absolute Error : 1050.7468664314322
R2 Score : 85.92978759337907


Best Model found : Lasso, R2_Score : 0.9368948574778251


4
